# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

### Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory (one string per line). For our source of company names, we can use either the list of restaurant names from the Restaurant Inspection dataset (see our prior session), and/or use a list of companies from a file available at https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt. We need to write Python code that will read these files and return a list of company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we need to write a function that takes as input two strings, and returns back their similarities (we will see below how to do that).
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.


### STEP 0: Fetch the data

In [4]:
# This is a list of banks
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt -o /home/ubuntu/data/bank-names.txt
    
# This is a list of NYC restaurants (also using this dataset for the regular expression notebooks)
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/uniquenames.txt -o /home/ubuntu/data/restaurant-names.txt   

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22747  100 22747    0     0  78716      0 --:--:-- --:--:-- --:--:-- 78982
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  364k  100  364k    0     0  1747k      0 --:--:-- --:--:-- --:--:-- 1751k


### STEP 1: Read the list of names from a file and create a list of names

In [1]:
# STEP 1: Read the list of names from a file and create a list of names
f = open("/home/ubuntu/data/restaurant-names.txt", "r")
content = f.read()
f.close()
lines = content.split("\n")

### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings (e.g., see [1](http://en.wikipedia.org/wiki/String_metric), [2](http://en.wikipedia.org/wiki/Approximate_string_matching), [3](http://en.wikipedia.org/wiki/Plagiarism_detection), [4](https://www.cs.cmu.edu/~pradeepr/papers/ijcai03.pdf), [5](http://en.wikipedia.org/wiki/Category:String_similarity_measures), and links within). 

For our example, we will use the q-gram similarity metric. What is a q-gram? Simply a sequency of q-consecutive characters in the string. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar. 

Hence, we can compute the similarity of two strings by first computing the set of q-grams for each string, and then compute the Jaccard coefficient between these sets. The [Jaccard coefficient](http://en.wikipedia.org/wiki/Jaccard_index) is simply defined as the size of the intersection divided by the size of the union of the two sets.

So, our task can be broken into two functions: 
* The first function takes a string and returns a list of the q-grams for the string, and 
* The second function takes as input two sets, and returns back their Jaccard coefficient.

In [5]:
# STEP 2: Now we implement our similarity function

# This returns a list of q-grams for a name
# getQgrams("Panos", 1) should return ["P", "a", "n", "s"]
# getQgrams("Panos", 2) should return ["Pa", "an", "no", "os"]
# etc
def getQgrams(name, q):
    # We add a little bit of padding of '#' characters
    # to create q-grams for the beginning and end of the 
    # string as well
    name = (q-1)*'#' + name + (q-1)*'#'
    length = len(name)
    qgrams = []
    for c in range(length):
        qgram = name[c:c+q]
        qgrams.append(qgram)
    return qgrams

# This function takes as input two names, computes their qgrams, and then computes
# the Jaccard coefficient (=intersection/union) of the two sets of qgrams
def jaccard(setA, setB):
    intersection = set(setA) & set(setB)
    union = set(setA) | set(setB)
    similarity = 1.0*len(intersection)/len(union)
    return similarity

def computeSimilarity(name1, name2, q):
    qgrams1 = getQgrams(name1.upper(), q)
    qgrams2 = getQgrams(name2.upper(), q)
    similarity = jaccard(qgrams1, qgrams2)
    return similarity

In [23]:
# Let's test our similarity function
print computeSimilarity("Microsoft", "Micrsoft", 3)
print computeSimilarity("Microsoft", "Micrsoft", 2)
print computeSimilarity("Microsoft", "Micrsoft", 1)
print computeSimilarity("Microsoft", "Google", 3)
print computeSimilarity("Microsoft", "Google", 2)
print computeSimilarity("Microsoft", "Google", 1)

0.666666666667
0.75
1.0
0.0952380952381
0.0555555555556
0.0909090909091


### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [17]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print data

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]


In [34]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a threshold parameter (by default set to be 0.5)
# that restricts the results to only string pairs with similarity
# above the threshold

# queryData finds similar names to a query
def queryData(query, companyList, threshold, q):
    result = []
    for r in companyList:
        # If the strings are equal, we do not add them in the results
        # as this is an uninteresting case
        if (r.upper()==query.upper()):
            continue
        # I will compute the similarity of the "query" string
        # with each restaurant "r", where "r" is the variable in the for loop
        similarity = computeSimilarity(query, r, q)
        # if the similarity is above 0.5, I will print the result
        if similarity>threshold:
            result.append( (r, similarity) )
    return result

def companySimilarity(companyName, companyList, threshold = 0.5, q = 3):
    results = queryData(companyName, companyList, threshold, q)
    results.sort(key=lambda tupl: tupl[1], reverse=True)
    return results


In [40]:
# This is the name of the restaurant that I am looking for
query = "McDonalds"
# This is the similarity configuration parameter
q = 2
threshold = 0.25
# lines contains the list of companiues
companySimilarity(query, lines, threshold, q)

[('MC DONALDS', 0.7692307692307693),
 ("MCDONALD'S", 0.7692307692307693),
 ('MCDONALD', 0.75),
 ("MC DONALD'S", 0.6),
 ("MCDONALD'S ", 0.6),
 ('MCDONALDS 17754', 0.5555555555555556),
 ('MCDONALDS (#11542)', 0.47619047619047616),
 ('MCDONALDS RESTAURANT', 0.43478260869565216),
 ('MCDONALD AVENUE DINER', 0.36),
 ("MCDONALD'S RESTAURANT", 0.36),
 ("MONA'S", 0.35714285714285715),
 ('MCDONALDS FAST FOOD RESTAURANT', 0.3225806451612903),
 ("MCDONALD'S (MACY'S 7TH FL)", 0.32142857142857145),
 ("MONACO'S", 0.3125),
 ("MCDONALD'S RESTAURANT #3391", 0.2903225806451613),
 ('DONNA', 0.2857142857142857),
 ('MONACO', 0.26666666666666666),
 ('ALL NATIONS', 0.2631578947368421),
 ('DORAL DONUTS', 0.2631578947368421),
 ('MRS. FIELDS', 0.2631578947368421)]

### Step 4: Perform the similarity computation across all companies in the dataset.

In [39]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.
q = 3
threshold = 0.5

# For brevity, we will do it for just the first 100 companies
for companyName in lines[:100]:
    matches = companySimilarity(companyName, lines, threshold, q)
    if len(matches)>0:
        print companyName, "===>"
        # we will print the top-5 matching names
        print matches[:5]
        print "================"

#1 SABOR LATINO RESTAURANT ===>
[('SABOR LATINO RESTAURANT', 0.78125), ('LATINO RESTAURANT', 0.59375), ('SABOR LATINO SEAFOOD RESTAURANT', 0.5853658536585366), ('EL REY LATINO RESTAURANT', 0.5405405405405406), ('RANCHO LATINO RESTAURANT', 0.5405405405405406)]
''U'' LIKE CHINESE RESTAURANT ===>
[('U LIKE CHINESE RESTAURANT', 0.7222222222222222), ('LEE CHINESE RESTAURANT', 0.5945945945945946), ('85 CHINESE RESTAURANT', 0.5675675675675675), ('88 CHINESE RESTAURANT', 0.5675675675675675), ('A+ CHINESE RESTAURANT', 0.5675675675675675)]
'WICHCRAFT ===>
[('WICHCRAFT', 0.6875)]
(LIBRARY)  FOUR & TWENTY BLACKBIRDS ===>
[('FOUR & TWENTY BLACKBIRDS', 0.6341463414634146)]
1001 NIGHTS ===>
[('1001 NIGHTS CAFE', 0.5909090909090909)]
1001 NIGHTS CAFE ===>
[('1001 NIGHTS', 0.5909090909090909)]
1005 CATERING ===>
[('GO CATERING', 0.5238095238095238)]
101 CAFE ===>
[('19 CAFE', 0.5333333333333333)]
101 RESTAURANT AND BAR ===>
[('508 RESTAURANT AND BAR', 0.6774193548387096), ('BESO RESTAURANT AND BAR', 0.